In [27]:
import pandas as pd
import csv

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('../data/politifact_plus.csv', usecols=['headline', 'target', 'references', 
                                                         'src_true','src_mostly_true', 'src_half_true', 'src_mostly_false', 'src_false', 'src_pants_on_fire'])
df.head()

,headline,target,references,src_true,src_mostly_true,src_half_true,src_mostly_false,src_false,src_pants_on_fire
0,"“The airport in Salzburg, Austria, has a count...",false,"instagram_post,the_local,washington_post,x_post",5,3,16,54,480,157
1,Video shows Palestinians pretending to be corp...,false,"instagram_post,instagram_post,youtube",4,13,35,53,764,340
2,The life span of a wind tower generator lasts ...,false,"none,x_post,facebook_post,interview,interview,...",24,50,108,247,1532,595
3,Matthew Perry died because of a COVID-19 vaccine.,false,"instagram_post,instagram_post,instagram_post,i...",5,3,16,54,480,157
4,A discrepancy in the number of ballots and vot...,false,"interview,interview,interview,new_york_citizen...",0,0,0,0,1,0


In [3]:
from clickbait import ClickbaitModel
from sentiment import SentimentModel
from spam import SpamModel
from source_reliable import SourceReliableModel

X_train, X_test, y_train, y_test = train_test_split(df[['headline', 'target', 'references', 
                                                        'src_true','src_mostly_true', 'src_half_true', 'src_mostly_false', 'src_false', 'src_pants_on_fire']], df['target'], test_size=.2, random_state=11)

clickM = ClickbaitModel()
sentiM = SentimentModel()
spamM = SpamModel()
srcM = SourceReliableModel()

sentiM.fit(X_train)
spamM.fit(X_train)
srcM.fit(X_train)


Training Accuracy: 0.951914
Testing Accuracy: 0.946875
Training Accuracy: 0.976010
Training Accuracy: 0.412616
Training Accuracy: 0.589594


In [13]:
acc_lst = [0.946, 0.976, 0.412, 0.589]
weight = [acc/sum(acc_lst) for acc in acc_lst]
clickbaitV = clickM.predict(X_test)[1] * weight[0]
sentiV = sentiM.predict(X_test)[1] * weight[1]
spamV = spamM.predict(X_test)[1] * weight[2]
sourceV = srcM.predict(X_test)[1] * weight[3]

In [28]:
train_clickbaitV = clickM.predict(X_train)[1] * weight[0]
train_sentiV = sentiM.predict(X_train)[1] * weight[1]
train_spamV = spamM.predict(X_train)[1] * weight[2]
train_sourceV = srcM.predict(X_train)[1] * weight[3]

In [36]:
veracity_train = pd.DataFrame({'clickbait':train_clickbaitV, 'sentiment': train_sentiV, 'spam': train_spamV, 'source': train_sourceV})
veracity_test = pd.DataFrame({'clickbait':clickbaitV, 'sentiment': sentiV, 'spam': spamV, 'source': sourceV})

In [37]:
clf = LogisticRegression(solver='liblinear')
clf.fit(veracity_train, y_train)
clf.score(veracity_test, y_test)

0.3047138047138047